In [35]:
import ass
import re
import os 
from os import path
import patoolib               #for unpacking
from pyunpack import Archive  # for unpacking
from shutil import copyfile
import pysrt
from mafan import simplify
from guess_language import guess_language
from itertools import compress

In [36]:
## get rid of all tags 
def replace_special(line_str):
    line_str = re.sub(r'\{.*\}','',line_str)
    return line_str 
def check_chinese(text):
    ch = re.findall(r'[\u4e00-\u9fff]+', text)
    return ch
def check_english(text):
    en = re.findall(r"[A-Za-z]+", text)
    return en

## use either gbk or big5 or utf-8
def open_ass(file):
    try:
        with open(file, "r",encoding="gbk") as f:
             doc = ass.parse(f)
        return doc 
    except:
        pass
    try:
        with open(file, "r",encoding="big5") as f:
             doc = ass.parse(f)
        return doc 
    except:
        pass
    try:
        with open(file, "r",encoding="utf-8") as f:
             doc = ass.parse(f)
        return doc 
    except:
        pass
    
    return None

def process_language(ele):
    try:
        text_list = ele.text.split('\\N')
        text_list = [replace_special(x) for x in text_list]
        languages = [guess_language(l) for l in text_list]
        en_index = [l=='en' for l in languages]
        zh_index = [l=='zh' for l in languages]
        en_lines = list(compress(text_list,en_index))
        zh_lines = list(compress(text_list,zh_index))
        if len(en_lines)==0 or len(zh_lines)==0:
            return None
        en_text = ' '.join(en_lines)
        if len(check_chinese(en_text)) > 0:
            return None 
        zh_text = simplify(' '.join(zh_lines))
        if len(check_english(zh_text))>0:
            return None
        text_line = en_text + ' | ' + zh_text
        return text_line
    except:
        return None
    
def validate_ass(file):
    subs = open_ass(file)
    if subs is not None:
        subs = subs.events
        languages = [process_language(ele) for ele in subs]
        return languages
    else:
        #print('Can not decode.')
        return None

In [31]:

data_ass = ["test.ass","test.ass"]


None
None


In [51]:
## search for srts with both chinese and english in it
## and transform them into txt, seperated by |, dump to results raw
#data_srts = [data_srt + f for f in os.listdir(data_srt)]
data_ass = ["test.ass","test.ass"]
results_raw = ""
counter=0
for ass_file in data_ass:
    res = validate_ass(ass_file)
    if res is not None: 
        total = len(res)
        #print(total) 
        if total > 0:
            none_values = sum([x is None for x in res ])
            ratio = float(none_values)/float(total)
            if ratio > 0.5:  
                pass         
            else:
                res_short = res[5:-5]
                res_short = [x for x in res_short if x is not None]
                counter+=1
                filename, file_extension = os.path.splitext(ass_file)
                fname = filename.split('/')[-1]
                fname = results_raw + str(counter)+'_'+fname+'.txt'
                with open(fname, mode='wt', encoding='utf-8') as myfile:
                    myfile.write('\n'.join(res_short))
                if counter%2 == 0:
                    print(counter)


2


2